In [1]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import torch

model_name = "ruslanmv/Medical-Llama3-8B"
device_map = 'cpu'

# bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",bnb_4bit_compute_dtype=torch.float16,)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True,use_cache=True,cache_dir="./llama3_model_dir",device_map=device_map)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def askme(question):
    sys_message = ''' 
    You are an AI Medical Assistant trained on a vast dataset of medicine information. Please be thorough and
    provide an precise answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    '''   
    # Create messages structured for the chat template
    messages = [{'role': 'system', 'content': sys_message}, {'role': 'user', 'content': question}]

    tokenizer.chat_template = """
{% for message in messages %}
{% if message['role'] == 'user' %}
User: {{ message['content'] + '\n' }}
{% else %}
Assistant: {{ message['content'] + '\n' }}
{% endif %}
{% endfor %}
"""

    # tokenizer.chat_template = "{% for message in messages %}{{message['role'] + '\n' + message['content'] + '\n'}}{% endfor %}"
    # Applying chat template
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
    outputs = model.generate(**inputs, max_new_tokens=100, use_cache=True)
    
    # Extract and return the generated text, removing the prompt
    response_text = tokenizer.batch_decode(outputs)[0].strip()
    answer = response_text.split('<|im_start|>assistant')[-1].strip()
    return answer
# Example usage
# - Context: First describe your problem.
# - Question: Then make the question.

question = '''I have throat cancer, can you suggest me some medicines as a beginning of my cancer-treatment.'''

print(askme(question))


/home/sagnik/anaconda3/envs/indicon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-29 03:36:05.464587: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 03:36:05.465782: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-29 03:36:05.483196: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 03:36:05.483213: E external/local_xla/xla/stream_executor/cuda/cuda_ff

[2024-11-29 03:36:06,869] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cpu (auto detect)


/home/sagnik/anaconda3/envs/indicon/lib/python3.10/site-packages/torch/distributed/distributed_c10d.py:266: UserWarning: Device capability of ccl unspecified, assuming `cpu` and `cuda`. Please specify it via the `devices` argument of `register_backend`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.38s/it]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Assistant:  
    You are an AI Medical Assistant trained on a vast dataset of medicine information. Please be thorough and
    provide an precise answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    

User: I have throat cancer, can you suggest me some medicines as a beginning of my cancer-treatment.

Assistant: Yes, I can. Here are some medicines that can help you with your throat cancer:

    1. Chemotherapy: This is a type of cancer treatment that uses drugs to kill cancer cells. It is usually given in a series of treatments over a period of time.
    2. Radiation therapy: This is a type of cancer treatment that uses high-energy rays to kill cancer cells. It is usually given in a series of treatments over a period of time.
    3.


In [2]:
question2 = 'New question. Can you tell me about the side effects of backlofen? Also provide some substitute medicines for it.'

print(askme(question2))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Assistant:  
    You are an AI Medical Assistant trained on a vast dataset of medicine information. Please be thorough and
    provide an precise answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    

User: New question. Can you tell me about the side effects of backlofen? Also provide some substitute medicines for it.

Assistant: 
    Backlofen is a nonsteroidal anti-inflammatory drug (NSAID) used to treat pain and inflammation. It is commonly used to treat pain caused by arthritis, muscle strains, and other injuries. However, like all medications, backlofen can cause side effects. Some of the most common side effects of backlofen include:
    
    * Stomach pain or upset stomach
    * Diarrhea
    * Nausea
    * Vomiting


In [3]:
question3="Also provide some substitute medicines for it."
print(askme(question3))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Assistant:  
    You are an AI Medical Assistant trained on a vast dataset of medicine information. Please be thorough and
    provide an precise answer. If you don't know the answer to a specific medical inquiry, advise seeking professional help.
    

User: Also provide some substitute medicines for it.

Assistant: Sure. Here are some common substitutes for the medicine you mentioned:

    Medicine: Aspirin
    Substitute: Ibuprofen, Acetaminophen, Naproxen

    Medicine: Paracetamol
    Substitute: Ibuprofen, Acetaminophen, Naproxen

    Medicine: Ibuprofen
    Substitute: Aspirin, Acetaminophen, Naproxen

    Medicine: Acetaminophen
    Substitute:
